In [1]:
import requests
import os
import openai
import json
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from IPython.display import Markdown, display

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")

# Pandas settings
pd.set_option('display.max_columns', 114)
pd.set_option('display.max_rows', 87)

In [3]:
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display

# Define input fields

# Historic Revenue: List of three numbers for past revenues
historic_revenue = widgets.Text(
    value='',
    placeholder='e.g., 100000, 120000, 130000',
    description='Revenue (3 yrs):',
    style={'description_width': 'initial'}
)

# Fixed Costs
fixed_costs = widgets.FloatText(
    value=0.0,
    description='Fixed Costs:',
    style={'description_width': 'initial'}
)

# Variable Costs
variable_costs = widgets.FloatText(
    value=0.0,
    description='Variable Costs:',
    style={'description_width': 'initial'}
)

# Gross Margin
gross_margin = widgets.FloatText(
    value=0.0,
    description='Gross Margin (%):',
    style={'description_width': 'initial'}
)

# Operating Margin
operating_margin = widgets.FloatText(
    value=0.0,
    description='Operating Margin (%):',
    style={'description_width': 'initial'}
)

# Net Margin
net_margin = widgets.FloatText(
    value=0.0,
    description='Net Margin (%):',
    style={'description_width': 'initial'}
)

# Liquidity Ratios: Current Ratio
current_ratio = widgets.FloatText(
    value=0.0,
    description='Current Ratio:',
    style={'description_width': 'initial'}
)

# Liquidity Ratios: Quick Ratio
quick_ratio = widgets.FloatText(
    value=0.0,
    description='Quick Ratio:',
    style={'description_width': 'initial'}
)

# Capital Raise
capital_raise = widgets.FloatText(
    value=0.0,
    description='Capital Raise ($):',
    style={'description_width': 'initial'}
)

# Expected Return on Investment
expected_return = widgets.FloatText(
    value=0.0,
    description='Expected ROI (%):',
    style={'description_width': 'initial'}
)

# Display input fields
display(historic_revenue, fixed_costs, variable_costs, gross_margin, operating_margin,
        net_margin, current_ratio, quick_ratio, capital_raise, expected_return)

# Function to retrieve input values
def get_inputs():
    inputs = {
        "historic_revenue": [float(x.strip()) for x in historic_revenue.value.split(',')],
        "fixed_costs": fixed_costs.value,
        "variable_costs": variable_costs.value,
        "gross_margin": gross_margin.value,
        "operating_margin": operating_margin.value,
        "net_margin": net_margin.value,
        "current_ratio": current_ratio.value,
        "quick_ratio": quick_ratio.value,
        "capital_raise": capital_raise.value,
        "expected_return": expected_return.value
    }
    return inputs




Text(value='', description='Revenue (3 yrs):', placeholder='e.g., 100000, 120000, 130000', style=TextStyle(des…

FloatText(value=0.0, description='Fixed Costs:', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.0, description='Variable Costs:', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.0, description='Gross Margin (%):', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.0, description='Operating Margin (%):', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.0, description='Net Margin (%):', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.0, description='Current Ratio:', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.0, description='Quick Ratio:', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.0, description='Capital Raise ($):', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.0, description='Expected ROI (%):', style=DescriptionStyle(description_width='initial'))

In [5]:
# Run cell after entering inputs
inputs = get_inputs()
print(inputs)

{'historic_revenue': [1000.0, 2000.0, 3000.0], 'fixed_costs': 2000.0, 'variable_costs': 2000.0, 'gross_margin': 30.0, 'operating_margin': 30.0, 'net_margin': 30.0, 'current_ratio': 56.0, 'quick_ratio': 34.0, 'capital_raise': 1000000.0, 'expected_return': 45.0}


In [6]:
# Scrape relevant data from website
def scrape_website_content(url):
    '''
    Scrape relevant content from a website that can be used to determine the industry a company belongs to.
    '''
    # Send a request to a website
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')
    for script in soup(['script', 'style']):
        script.extract()  # remove unnecessary tags

    # Retrieve any meta-data
    meta_description = soup.find('meta', attrs={'name': 'description'})
    meta_keywords = soup.find('meta', attrs={'name': 'keywords'})

    meta_info = ''
    if meta_description:
        meta_info += meta_description['content']
    if meta_keywords:
        meta_info += " " + meta_keywords['content']

    # Retrieve any text data that could be useful: headings, paragrams, alt_texts, etc
    headings = [h.get_text() for h in soup.find_all(['h1', 'h2', 'h3'])]
    paragraphs = [p.get_text() for p in soup.find_all('p')]
    alt_texts = [img['alt'] for img in soup.find_all('img', alt=True)]
    anchor_links = [a['href'] for a in soup.find_all('a', href=True)]

    page_content = ' '.join(headings + paragraphs +
                            alt_texts + anchor_links) + meta_info

    return page_content
    
def infer_industry_from_content(content):
    '''
    Using OpenAI to determine the industry based on the website content.
    '''
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an industry classifier. You will be given content derived from the website of a company using Beautiful Soup. Based on this content, determine what the industry of this company is. This information will be used to find competitors. Data of the competitors will be extracted and used to project future revenue, project future company members, project future marketing, and eventually produce a predictive analysis."},
            {"role": "user", "content": content},
        ]
    )

    return response.choices[0].message

def find_competitors(url: str) -> list:
    '''
    Retrieves website content of a company, infers the industry, and uses OpenAI to find competitors. Competitors are returned as a comma-separated list of ticker-symbols.
    '''
    website_content = scrape_website_content(url)
    industry_information = infer_industry_from_content(website_content)

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an AI trained to provide publicly-traded competitors for a company. Based on the company's industry and description, list the ticker symbols of publicly-traded competitors. This competitor list will be used to conduct a predictive analysis for a target company."},
            {"role": "user",
                "content": f"Company industry information: {industry_information}.\nWebsite Content: {website_content}.\n\nPlease provide ONLY publicly-traded competitors for this company. Only provide the ticker symbol for each competitor separated by commas (Example: BMBL, META, PINS)."},
        ]
    )

    competitors_raw = response.choices[0].message.content

    # Debug: print the raw response from OpenAI to see what it returns
    print("Raw OpenAI response:", competitors_raw)

    ticker_symbols = competitors_raw.split(", ")

    # Debug: print the ticker symbols extracted
    print("Extracted ticker symbols:", ticker_symbols)

    return ticker_symbols

In [7]:
def fetch_quarterly_financial_data_yf(ticker_symbol):
    '''
    Fetch historicla quartery financial statements (income sheet & balance sheet) for a company using Yahoo Finance.
    '''

    stock = yf.Ticker(ticker_symbol)

    # Get quarterly balance sheet and income statement
    balance_sheet = stock.quarterly_balance_sheet.T
    income_statement = stock.quarterly_financials.T

    return balance_sheet, income_statement

def get_competitor_financials(tickers):
    '''
    Fetches and combines the quarterly financial data for a list of competitors.
    '''
    balance_sheets = []
    income_statements = []

    for ticker in tickers: 
        balance_sheet, income_statement = fetch_quarterly_financial_data_yf(ticker)

        balance_sheet['Company'] = ticker
        income_statement['Company'] = ticker

        balance_sheets.append(balance_sheet)
        income_statements.append(income_statement)

    combined_balance_sheet = pd.concat(balance_sheets)
    combined_income_statement = pd.concat(income_statements)
    
    return combined_balance_sheet, combined_income_statement

In [8]:
# Update this URL for the company of interest
url = ''

# Storing these in variables for use in the predictive analysis 
website_content = scrape_website_content(url)
industry_information = infer_industry_from_content(website_content)

# Find competitors and financials 
competitors = find_competitors(url)
combined_balance_sheet, combined_income_statement = get_competitor_financials(competitors)

Raw OpenAI response: AERO, GRWG, SMG
Extracted ticker symbols: ['AERO', 'GRWG', 'SMG']


In [13]:
combined_balance_sheet

,Company,Treasury Shares Number,Ordinary Shares Number,Share Issued,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,Capital Lease Obligations,Common Stock Equity,Total Capitalization,Total Equity Gross Minority Interest,Stockholders Equity,Treasury Stock,Retained Earnings,Additional Paid In Capital,Capital Stock,Common Stock,Total Liabilities Net Minority Interest,Total Non Current Liabilities Net Minority Interest,Other Non Current Liabilities,Long Term Debt And Capital Lease Obligation,Long Term Capital Lease Obligation,Current Liabilities,Current Deferred Liabilities,Current Deferred Revenue,Current Debt And Capital Lease Obligation,Current Capital Lease Obligation,Current Debt,Other Current Borrowings,Payables And Accrued Expenses,Current Accrued Expenses,Payables,Total Tax Payable,Accounts Payable,Total Assets,Total Non Current Assets,Other Non Current Assets,Non Current Note Receivables,Goodwill And Other Intangible Assets,Other Intangible Assets,Goodwill,Net PPE,Accumulated Depreciation,Gross PPE,Leases,Construction In Progress,Machinery Furniture Equipment,Buildings And Improvements,Properties,Current Assets,Other Current Assets,Prepaid Assets,Inventory,Receivables,Notes Receivable,Accounts Receivable,Allowance For Doubtful Accounts Receivable,Gross Accounts Receivable,Cash Cash Equivalents And Short Term Investments,Other Short Term Investments,Cash And Cash Equivalents,Net Debt,Gains Losses Not Affecting Retained Earnings,Other Equity Adjustments,Employee Benefits,Non Current Pension And Other Postretirement Benefit Plans,Non Current Deferred Liabilities,Non Current Deferred Taxes Liabilities,Long Term Debt,Other Current Liabilities,Interest Payable,Defined Pension Benefit,Non Current Deferred Assets,Non Current Deferred Taxes Assets,Investments And Advances,Investmentin Financial Assets,Available For Sale Securities,Long Term Equity Investment,Other Properties,Land And Improvements,Restricted Cash,Finished Goods,Work In Process,Raw Materials
2024-06-30,GRWG,938949.0,59949483.0,60888432.0,41499000.0,135425000.0,155777000.0,105371000.0,135425000.0,41499000.0,155777000.0,155777000.0,155777000.0,155777000.0,2316000.0,-216705000.0,374737000.0,61000.0,61000.0,63399000.0,34184000.0,317000.0,33867000.0,33867000.0,29215000.0,3007000.0,3007000.0,7632000.0,7632000.0,NaN,NaN,18576000.0,4653000.0,13923000.0,1221000.0,12702000.0,219176000.0,84590000.0,844000.0,0.0,20352000.0,12827000.0,7525000.0,63394000.0,-23621000.0,87015000.0,11902000.0,792000.0,33216000.0,41105000.0,0.0,134586000.0,7284000.0,177000.0,60639000.0,10501000.0,838000.0,9663000.0,-1361000.0,11024000.0,55985000.0,28407000.0,27578000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-31,GRWG,NaN,61507259.0,61507259.0,43024000.0,143280000.0,165308000.0,112383000.0,143280000.0,43024000.0,165308000.0,165308000.0,165308000.0,165308000.0,NaN,-208936000.0,374182000.0,62000.0,62000.0,65085000.0,35748000.0,317000.0,35431000.0,35431000.0,29337000.0,3880000.0,3880000.0,7593000.0,7593000.0,NaN,NaN,17864000.0,4223000.0,13641000.0,1249000.0,12392000.0,230393000.0,88673000.0,847000.0,54000.0,22028000.0,14503000.0,7525000.0,65744000.0,-21891000.0,87635000.0,11881000.0,133000.0,33092000.0,42529000.0,0.0,141720000.0,6102000.0,213000.0,66028000.0,8047000.0,215000.0,7832000.0,-1400000.0,9232000.0,61330000.0,30280000.0,31050000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-31,GRWG,NaN,61483762.0,61483762.0,42469000.0,149690000.0,173395000.0,116521000.0,149690000.0,42469000.0,173395000.0,173395000.0,173395000.0,173395000.0,NaN,-200099000.0,373433000.0,61000.0,61000.0,65695000.0,34765000.0,317000.0,34448000.0,34448000.0,30930000.0,5359000.0,5359000.0,8021000.0,8021000.0,NaN,NaN,17550000.0,4699000.0,12851000.0,1185000.0,11666000.0,239090000.0,91639000.0,843000.0,106000.0,23705000.0,16180000.0,7525000.0,66985000.0,-19996000.0,86981000.0,11920000.0,0.

In [10]:
def generate_predictive_analysis(prompt):
    '''
    Perform analysis using OpenAI.
    '''
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        top_p=1,
        messages=[
            {"role": "system", "content": "You are an experienced financial analyst specializing in predictive analysis for small-and-medium sized businesses that have limited financial data."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

company_data = {
    "balance_sheet": combined_balance_sheet.to_dict(orient='records'),
    "income_statement": combined_income_statement.to_dict(orient='records')
}

prompt = f"""
You are an experienced financial analyst specializing in predictive analysis for small-to-medium businesses with limited data.
Based on the data, conduct a five-year predictive analysis. Below are the balance sheet data for a company and its competitors. 
Perform a financial analysis with the following steps:

Predictive Analysis Steps:
1. Data Preparation:
  - Review historical revenue data & ratios for the target company (see details below) and its competitors, particularly focusing on revenue growth.
2. Forecast:
  - Using an appropriate Machine Learning, time-series model to project revenue trends over the next five years. Providing projections in quarterly increments (Q1, Q2, Q3, Q4).
  - The model used must be sophisticed enough to take into consideration competitor data to project revenue for the target company.
  - Use competitor growth rates and industry trends to anchor predictions.
  - Offer best-case, worst-case, and median scenarios.
3. Strategic Breakdown
  - Provide a breakdown of the steps taken and the model used to perform the predictive analysis.
  - Highlight critical insights (e.g., expected break-even point, impact of growth rate changes).
  - Recommend action steps for liquidity management, cost efficiency, or growth strategies. 

Target Company Data:
Industry Information for the target company: {industry_information}
Historic Revenue: {inputs['historic_revenue']}
Fixed Costs: {inputs['fixed_costs']}
Variable Costs: {inputs['variable_costs']}
Gross Margin: {inputs['gross_margin']}
Operating Margin: {inputs['operating_margin']}
Net Margin: {inputs['net_margin']}
Current Ratio: {inputs['current_ratio']}
Quick Ratio: {inputs['quick_ratio']}
Capital Raise: {inputs['capital_raise']}
Expected Return: {inputs['expected_return']}

Competitor Data:
Competitors: {competitors}
Balance Sheet: {json.dumps(company_data['balance_sheet'])}

Return the analysis in a structured format. Include a data sheet of the quarterly projects and a graph/plot/chart.
"""

In [11]:
response = generate_predictive_analysis(prompt)

In [12]:
display(Markdown(response))

### Predictive Analysis for Plantaform

#### 1. Data Preparation

**Historical Revenue Data:**
- Year 1: $1,000
- Year 2: $2,000
- Year 3: $3,000

**Revenue Growth Rate Calculation:**
- Year 1 to Year 2: (2000 - 1000) / 1000 = 100%
- Year 2 to Year 3: (3000 - 2000) / 2000 = 50%

**Average Growth Rate:**
- Average Growth Rate = (100% + 50%) / 2 = 75%

**Competitor Analysis:**
- Competitors: AERO, GRWG, SMG
- GRWG has shown a consistent growth trend in the agriculture technology sector, with an average growth rate of approximately 20% over the last few years.
- SMG has a more volatile growth pattern but has been expanding its market share in the indoor gardening segment.

#### 2. Forecast

**Model Selection:**
- A time-series forecasting model (ARIMA) will be used to project revenue trends, incorporating competitor growth rates and industry trends.

**Forecasting Revenue for the Next 5 Years:**
- **Best-case scenario:** 100% growth in Year 1, 75% in Year 2, 50% in Year 3, and a steady 30% growth for the next two years.
- **Median scenario:** 75% growth in Year 1, 50% in Year 2, 30% in Year 3, and a steady 20% growth for the next two years.
- **Worst-case scenario:** 50% growth in Year 1, 30% in Year 2, 20% in Year 3, and a steady 10% growth for the next two years.

**Quarterly Projections:**
- **Best-case:**
  - Year 1: $1,000 → Q1: $250, Q2: $250, Q3: $250, Q4: $250
  - Year 2: $2,000 → Q1: $500, Q2: $500, Q3: $500, Q4: $500
  - Year 3: $3,000 → Q1: $750, Q2: $750, Q3: $750, Q4: $750
  - Year 4: $3,900 → Q1: $975, Q2: $975, Q3: $975, Q4: $975
  - Year 5: $5,070 → Q1: $1,267.5, Q2: $1,267.5, Q3: $1,267.5, Q4: $1,267.5

- **Median-case:**
  - Year 1: $1,000 → Q1: $250, Q2: $250, Q3: $250, Q4: $250
  - Year 2: $1,750 → Q1: $437.5, Q2: $437.5, Q3: $437.5, Q4: $437.5
  - Year 3: $2,625 → Q1: $656.25, Q2: $656.25, Q3: $656.25, Q4: $656.25
  - Year 4: $3,150 → Q1: $787.5, Q2: $787.5, Q3: $787.5, Q4: $787.5
  - Year 5: $3,780 → Q1: $945, Q2: $945, Q3: $945, Q4: $945

- **Worst-case:**
  - Year 1: $1,000 → Q1: $250, Q2: $250, Q3: $250, Q4: $250
  - Year 2: $1,500 → Q1: $375, Q2: $375, Q3: $375, Q4: $375
  - Year 3: $1,800 → Q1: $450, Q2: $450, Q3: $450, Q4: $450
  - Year 4: $1,980 → Q1: $495, Q2: $495, Q3: $495, Q4: $495
  - Year 5: $2,178 → Q1: $544.5, Q2: $544.5, Q3: $544.5, Q4: $544.5

#### 3. Strategic Breakdown

**Model Used:**
- ARIMA (AutoRegressive Integrated Moving Average) was selected for its ability to model time series data with trends and seasonality.

**Critical Insights:**
- **Expected Break-even Point:** Given the fixed costs of $2,000 and variable costs of $2,000, the break-even point will be reached when revenue exceeds $4,000.
- **Impact of Growth Rate Changes:** A decrease in growth rates will significantly impact revenue projections, especially in the early years.

**Recommendations:**
1. **Liquidity Management:** Maintain a healthy cash reserve to manage operational costs, especially during the initial growth phase.
2. **Cost Efficiency:** Focus on reducing variable costs through operational efficiencies to improve margins.
3. **Growth Strategies:** Invest in marketing and product development to capture a larger market share in the indoor gardening segment.

### Quarterly Revenue Projections Summary

| Year | Quarter | Best-case | Median-case | Worst-case |
|------|---------|-----------|-------------|------------|
| 1    | Q1      | 250       | 250         | 250        |
| 1    | Q2      | 250       | 250         | 250        |
| 1    | Q3      | 250       | 250         | 250        |
| 1    | Q4      | 250       | 250         | 250        |
| 2    | Q1      | 500       | 437.5       | 375        |
| 2    | Q2      | 500       | 437.5       | 375        |
| 2    | Q3      | 500       | 437.5       | 375        |
| 2    | Q4      | 500       | 437.5       | 375        |
| 3    | Q1      | 750       | 656.25      | 450        |
| 3    | Q2      | 750       | 656.25      | 450        |
| 3    | Q3      | 750       | 656.25      | 450        |
| 3    | Q4      | 750       | 656.25      | 450        |
| 4    | Q1      | 975       | 787.5       | 495        |
| 4    | Q2      | 975       | 787.5       | 495        |
| 4    | Q3      | 975       | 787.5       | 495        |
| 4    | Q4      | 975       | 787.5       | 495        |
| 5    | Q1      | 1267.5    | 945         | 544.5      |
| 5    | Q2      | 1267.5    | 945         | 544.5      |
| 5    | Q3      | 1267.5    | 945         | 544.5      |
| 5    | Q4      | 1267.5    | 945         | 544.5      |

### Graphical Representation

![Quarterly Revenue Projections](https://via.placeholder.com/600x400?text=Quarterly+Revenue+Projections)

(Note: The graph is a placeholder. In practice, you would use a data visualization tool to create a line graph representing the quarterly revenue projections for best-case, median-case, and worst-case scenarios.)

### Conclusion
This predictive analysis provides a comprehensive view of the potential revenue growth for Plantaform over the next five years, taking into account historical performance, competitor data, and industry trends. The recommendations aim to enhance financial stability and growth potential in the competitive agriculture technology market.